In [12]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [2]:
pd.set_option("display.max_columns", 100)

In [3]:
analysis_date = dt.date(2022,5,25)

In [4]:
itp_id = 300

In [5]:
#route = 3479

In [33]:
df = (tbl.views.gtfs_schedule_fact_daily_trips()
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
      >> collect()
)

In [39]:
df

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
0,4100255323842349957,5463476953836023854,882540,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,14,14,66720,68040,0.366667
1,4100255323842349957,8085206343096106202,882534,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,14,14,61860,63240,0.383333
2,4100255323842349957,-9125883904854161573,883680,3501,300,0,40510,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,17,17,38280,39120,0.233333
3,4100255323842349957,2337859805703098145,883319,3495,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,17,17,49680,52500,0.783333
4,4100255323842349957,-856925526770901337,883707,3501,300,0,77710,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,17,17,59400,60780,0.383333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511,4100255323842349957,-6344409580421376528,881848,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,53,53,36120,39771,1.014167
1512,4100255323842349957,7075752003393539302,881931,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,54,54,24480,27960,0.966667
1513,4100255323842349957,-6488472625415052037,881893,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,54,54,60300,64260,1.100000
1514,4100255323842349957,2630782785983138556,881902,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,54,54,52440,56340,1.083333


In [34]:
(df
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
 >> filter(_.calitp_extracted_at <= analysis_date, _.calitp_deleted_at >= analysis_date)
 >> summarize(unique= _.trip_id.nunique())
      
)

,unique
0,1516


In [35]:
(df
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
 >> filter(_.calitp_extracted_at <= analysis_date, _.calitp_deleted_at >= analysis_date)
 >> group_by(_.route_id) 
 >> summarize(n=_.trip_id.nunique()) >> arrange(-_.n))

,route_id,n
0,3479,180
4,3485,153
8,3489,153
2,3481,145
9,3490,107
18,3501,104
12,3493,88
1,3480,82
15,3496,76
5,3486,71


In [45]:
trips =(tbl.views.gtfs_schedule_dim_trips()
 >>filter(_.calitp_itp_id==itp_id) >> collect())


In [46]:
trips

,calitp_itp_id,calitp_url_number,route_id,service_id,trip_id,shape_id,trip_headsign,trip_short_name,direction_id,block_id,wheelchair_accessible,bikes_allowed,calitp_extracted_at,calitp_hash,trip_key,calitp_deleted_at
0,300,0,3319,20210411_10,854726,25282,UCLA,None,0,None,0,0,2021-07-27,jXl4q7vsDbO02R199qEzDw==,2893768055670690004,2021-09-01
1,300,0,3305,20210411_2,852558,25235,Downtown SM,None,1,None,0,0,2021-07-27,BfhPR7FrJIxIuh3+0stJkw==,-6472242146536274745,2021-09-01
2,300,0,3348,10,858661,25384,San Vicente Blvd & 14th St,None,0,104006,0,0,2021-10-11,wLNklyXk8rHhTvez02TbWA==,-5004370816273597550,2021-10-19
3,300,0,3340,10,858137,25362,Bundy Station,None,1,None,0,0,2021-09-01,SUYOnlRGi3O7dC0m9ZqIUA==,7664025582092981070,2021-10-11
4,300,0,3315,20210411_10,854346,25267,Brentwood,None,0,None,0,0,2021-07-27,02xAWu+T82PNDm6Uo4qmnw==,2388779801839159418,2021-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41265,300,0,3490,10,882851,25921,Playa Vista,None,1,None,0,0,2022-03-29,YsUpz2yqq6qsjKs6u23ASA==,7899348205952208772,2099-01-01
41266,300,0,3490,1,882686,25921,Playa Vista,None,1,None,0,0,2022-03-29,ER4bboU7Mt9eBkyWLiqVoQ==,-1943958898270726548,2099-01-01
41267,300,0,3490,10,882884,25921,Playa Vista,None,1,None,0,0,2022-03-29,pp2gzfcOuBfFsZvj8GroVw==,-1186431545781379683,2099-01-01
41268,300,0,3490,1,882720,25921,Playa Vista,None,1,None,0,0,2022-03-29,diLSHIw0JJWP30rAqXUtbg==,-3734102812423318981,2099-01-01


In [47]:
pd.merge(df, trips, on="route_id")

,feed_key,trip_key_x,trip_id_x,route_id,calitp_itp_id_x,calitp_url_number_x,service_id_x,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at_x,calitp_deleted_at_x,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,service_id_y,trip_id_y,shape_id,trip_headsign,trip_short_name,direction_id,block_id,wheelchair_accessible,bikes_allowed,calitp_extracted_at_y,calitp_hash,trip_key_y,calitp_deleted_at_y
0,4100255323842349957,5463476953836023854,882540,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,14,14,66720,68040,0.366667,300,0,20220327_1,882338,25914,UCLA Rapid,None,0,None,0,0,2022-03-08,zOL6TnpETqoR90vnFdmMJw==,6931924934895239968,2022-03-29
1,4100255323842349957,5463476953836023854,882540,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,14,14,66720,68040,0.366667,300,0,20220327_10,882539,25914,UCLA Rapid,None,0,None,0,0,2022-03-08,mOpMyZPr1gWdMWM3TZTHGg==,-687299595928855089,2022-03-29
2,4100255323842349957,5463476953836023854,882540,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,14,14,66720,68040,0.366667,300,0,20220327_1,882342,25914,UCLA Rapid,None,0,None,0,0,2022-03-08,LwbXXJz+UZYru/ZAn4sSHg==,-3655034280603836249,2022-03-29
3,4100255323842349957,5463476953836023854,882540,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,14,14,66720,68040,0.366667,300,0,20220327_1,882349,25914,UCLA Rapid,None,0,None,0,0,2022-03-08,Ivk410v8QPgkgJuoYVC1MQ==,-8488052539707252998,2022-03-29
4,4100255323842349957,5463476953836023854,882540,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,14,14,66720,68040,0.366667,300,0,20220327_10,882559,25914,UCLA Rapid,None,0,None,0,0,2022-03-08,s4so0AmNi3kzscuoeaGFgw==,-3619909437016100009,2022-03-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735735,4100255323842349957,-4866577048245157158,882322,3488,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,32,32,25200,29363,1.156389,300,0,20220327_10,882321,25911,Downtown LA Express,None,0,None,0,0,2022-03-08,YllGDf/vHLargMZ7D5xA1w==,-3355637690917848168,2022-03-29
735736,4100255323842349957,-4866577048245157158,882322,3488,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,32,32,25200,29363,1.156389,300,0,20220327_10,882322,25911,Downtown LA Express,None,0,None,0,0,2022-03-08,D8BMDdOJPcdo31KlLc8Txw==,5846187615158265287,2022-03-29
735737,4100255323842349957,-4866577048245157158,882322,3488,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,32,32,25200,29363,1.156389,300,0,20220327_10,882325,25912,Santa Monica Express,None,1,None,0,0,2022-03-08,n1HDPgRsD9JF+Ad4/kKnfg==,575061644207474656,2022-03-29
735738,4100255323842349957,-4866577048245157158,882322,3488,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,32,32,25200,29363,1.156389,300,0,10,882324,25912,Santa Monica Express,None,1,None,0,0,2022-03-29,M1j/36f+RufQjmZR97GT6w==,4352168358434773719,2099-01-01


In [8]:
# (tbl.views.gtfs_schedule_fact_daily_trips()
#  >>filter(_.calitp_itp_id==itp_id)
#  >>filter(_.service_date == analysis_date)
# )

In [9]:
# (
#     tbl.views.gtfs_rt_fact_daily_feeds()
#     >> filter(_.calitp_itp_id == itp_id)
#     >> arrange(_.date ==analysis_date)
# )

In [10]:
# airtable_organizations = (
#     tbl.airtable.california_transit_organizations()
#     >> select(_.itp_id, _.name, _.caltrans_district,
#               _.website, _.ntp_id, _.drmt_organization_name)
#     >> filter(_.itp_id == itp_id)
#     >> collect()
# )

In [11]:
# airtable_organizations